In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
import os
import sys

In [6]:
promptNER_strcure = {"definition":"An entity is",
                     "examplaires":[],
                     "dispute_task_exemplars":[],
                     "response":[],
                     "questions":"Q: Given the paragraph below, identify a list of entities and for each entry explain why it is an entity. Answer in the format: \n",
                     "dispute_task":"Q: Given the paragraph below, the entity phrase and two proposed entity types, identify what the actual type of the entity is \nParagraph:",
                     "paragraph":""
                     }

In [8]:
class Config:
    cot_format = """
    Format: 
    
    1. First Candidate | True | Explanation why the word is an entity (entity_type)
    2. Second Candidate | False | Explanation why the word is not an entity (entity_type)
    """

    no_tf_format = """
    1. First Entity | Explanation why the word is an entity (entity_type)
    2. Second Entity | Explanation why the word is not an entity (entity_type)
    """

    tf_format = """
    Format: 

    1. First Candidate | True | (entity_type)
    2. Second Candidate | False | (entity_type)
    """

    exemplar_format = """
    Format:    
    
    1. First Entity | (entity_type)
    2. Second Entity | (entity_type)
    """

    
#class promptNERConfig()

In [9]:
class ConllConfig(Config):
    defn = "An entity is a person (PER), title, named organization (ORG), location (LOC), country (LOC) or nationality (MISC)." \
           "Names, first names, last names, countries are entities. Nationalities are entities even if they are " \
           "adjectives. Sports, sporting events, adjectives, verbs, numbers, " \
                "adverbs, abstract concepts, sports, are not entities. Dates, years and times are not entities. " \
           "Possessive words like I, you, him and me are not entities. " \
           "If a sporting team has the name of their location and the location is used to refer to the team, " \
           "it is an entity which is an organisation, not a location"

    defn = "An entity is a person (PER), title, named organization (ORG), location (LOC), country (LOC) or nationality (MISC)."

    cot_exemplar_1 = """
    After bowling Somerset out for 83 on the opening morning at Grace Road , Leicestershire extended their first innings by 94 runs before being bowled out for 296 with England discard Andy Caddick taking three for 83 .
    
    Answer:
    1. bowling | False | as it is an action
    2. Somerset | True | Somerset is used as a sporting team here, not a location hence it is an organisation (ORG)
    3. 83 | False | as it is a number 
    4. morning | False| as it represents a time of day, with no distinct and independant existence
    5. Grace Road | True | the game is played at Grace Road, hence it is a place or location (LOC)
    6. Leicestershire | True | is the name of a cricket team that is based in the town of Leicestershire, hence it is an organisation (ORG). 
    7. first innings | False | as it is an abstract concept of a phase in play of cricket
    8. England | True | as it is a place or location (LOC)
    9. Andy Caddick | True | as it is the name of a person. (PER) 
    """
    cot_exemplar_2 = """
    Their stay on top , though , may be short-lived as title rivals Essex , Derbyshire and Surrey all closed in on victory while Kent made up for lost time in their rain-affected match against Nottinghamshire .
    
    Answer:
    1. Their | False | as it is a possessive pronoun
    2. stay | False | as it is an action
    3. title rivals | False | as it is an abstract concept
    4. Essex | True |  Essex are title rivals is it a sporting team organisation not a location (ORG)
    5. Derbyshire | True |  Derbyshire are title rivals is it a sporting team organisation not a location (ORG)
    6. Surrey | True |  Surrey are title rivals is it a sporting team organisation not a location (ORG)
    7. victory | False | as it is an abstract concept
    8. Kent | True |  Kent lost to Nottinghamshire, it is a sporting team organisation not a location (ORG)
    9. Nottinghamshire | True |  Kent lost to Nottinghamshire, it is a sporting team organisation not a location (ORG)
    
    """

    cot_exemplar_3 = """
    But more money went into savings accounts , as savings held at 5.3 cents out of each dollar earned in both June and July .
    
    Answer:
    1. money | False | as it is not a named person, organization or location
    2. savings account | False | as it is not a person, organization or location
    3. 5.3 | False | as it is a number
    4. June | False | as it is a date
    5. July | False | as it is a date
    """

    cot_exemplars = [cot_exemplar_1, cot_exemplar_2, cot_exemplar_3]

    no_tf_exemplar_1 = """
        After bowling Somerset out for 83 on the opening morning at Grace Road , Leicestershire extended their first innings by 94 runs before being bowled out for 296 with England discard Andy Caddick taking three for 83 .
        
        Answer:
        1. Somerset | Somerset is used as a sporting team here, not a location hence it is an organisation (ORG)
        2. Grace Road | the game is played at Grace Road, hence it is a place or location (LOC)
        3. Leicestershire | is the name of a cricket team that is based in the town of Leicestershire, hence it is an organisation (ORG). 
        4. England | as it is a place or location (LOC)
        5. Andy Caddick | as it is the name of a person. (PER) 
        """
    no_tf_exemplar_2 = """
        Their stay on top , though , may be short-lived as title rivals Essex , Derbyshire and Surrey all closed in on victory while Kent made up for lost time in their rain-affected match against Nottinghamshire .
        
        Answer:
        1. Essex | since Essex are title rivals is it a sporting team organisation not a location (ORG)
        2. Derbyshire | since Derbyshire are title rivals is it a sporting team organisation not a location (ORG)
        3. Surrey | since Surrey are title rivals is it a sporting team organisation not a location (ORG)
        4. Kent | since Kent lost to Nottinghamshire, it is a sporting team organisation not a location (ORG)
        5. Nottinghamshire | since Kent lost to Nottinghamshire, it is a sporting team organisation not a location (ORG)
        """

    no_tf_exemplar_3 = """
        But more money went into savings accounts , as savings held at 5.3 cents out of each dollar earned in both June and July .

        Answer:
        1. 

        """
    no_tf_exemplars = [no_tf_exemplar_1, no_tf_exemplar_2, no_tf_exemplar_3]

    tf_exemplar_1 = """
        After bowling Somerset out for 83 on the opening morning at Grace Road , Leicestershire extended their first innings by 94 runs before being bowled out for 296 with England discard Andy Caddick taking three for 83 .

        Answer:
        1. bowling | False | None 
        2. Somerset | True | (ORG)
        3. 83 | False | None
        4. morning | False | None
        5. Grace Road | True | (LOC)
        6. Leicestershire | True | (ORG)
        7. first innings | False | None
        8. England | True | (LOC)
        9. Andy Caddick | True | (PER)
        """
    tf_exemplar_2 = """
        Their stay on top , though , may be short-lived as title rivals Essex , Derbyshire and Surrey all closed in on victory while Kent made up for lost time in their rain-affected match against Nottinghamshire .

        Answer:
        1. Their | False | None
        2. stay | False | None
        3. title rivals | False | None
        4. Essex | True | (ORG)
        5. Derbyshire | True | (ORG)
        6. Surrey | True | (ORG)
        7. victory | False | None
        8. Kent | True | (ORG)
        9. Nottinghamshire | True | (ORG)

        """

    tf_exemplar_3 = """
        But more money went into savings accounts , as savings held at 5.3 cents out of each dollar earned in both June and July .

        Answer:
        1. money | False | None
        2. savings account | False | None
        3. 5.3 | False | None
        4. June | False | None
        5. July | False | None

        """
    tf_exemplars = [tf_exemplar_1, tf_exemplar_2, tf_exemplar_3]

    exemplar_1 = """
        After bowling Somerset out for 83 on the opening morning at Grace Road , Leicestershire extended their first innings by 94 runs before being bowled out for 296 with England discard Andy Caddick taking three for 83 .
        
        Answer:
        1. Somerset | (ORG)
        2. Grace Road | (LOC)
        3. Leicestershire | (ORG). 
        4. England | (LOC)
        5. Andy Caddick | (PER) 
    """
    exemplar_2 = """
        Their stay on top , though , may be short-lived as title rivals Essex , Derbyshire and Surrey all closed in on victory while Kent made up for lost time in their rain-affected match against Nottinghamshire .
        
        Answer:
        1. Essex | (ORG)
        2. Derbyshire | (ORG)
        3. Surrey | (ORG)
        4. Kent | (ORG)
        5. Nottinghamshire | (ORG)
    """

    exemplar_3 = """
    But more money went into savings accounts , as savings held at 5.3 cents out of each dollar earned in both June and July .

    Answer:
    1. 
    """
    exemplars = [exemplar_1, exemplar_2, exemplar_3]

    type_exemplar_1 = """
    After bowling Somerset out for 83 on the opening morning at Grace Road , Leicestershire extended their first innings by 94 runs before being bowled out for 296 with England discard Andy Caddick taking three for 83 .

    Entity Phrase: Somerset
    Answer: Somerset is used as a sporting team here, not a location hence it is an organisation (ORG)
    
    Entity Phrase: England
    Answer: England is a country hence it is a location (LOC)
    
    Entity Phrase: Grace Road
    Answer: at Grace Road indicates this is a location or venue (LOC)    
    """

    type_exemplar_2 = """
    Their stay on top , though , may be short-lived as title rivals Essex , Derbyshire and Surrey all closed in on victory while Kent made up for lost time in their rain-affected match against Nottinghamshire .
    
    Entity Phrase: Essex
    Answer: As they are tital rivals, Essex is a sports team and not a location (ORG)
    
    Entity Phrase: Nottinghamshire
    Answer: As Nottinghamshire defeated Kent, this is a sports team not a location (ORG)
    """
    type_exemplars = [type_exemplar_1, type_exemplar_2]

    dispute_exemplar_1 = """
    After bowling Somerset out for 83 on the opening morning at Grace Road , Leicestershire extended their first innings by 94 runs before being bowled out for 296 with England discard Andy Caddick taking three for 83 .

    Entity Phrase: Somerset, Options: [(LOC), (ORG)]
    Answer: Somerset is used as a sporting team here, not a location hence it is an organisation (ORG)

    Entity Phrase: England, Options: [(LOC), (PER)]
    Answer: England is a country hence it is a location not a person (LOC)

    Entity Phrase: Grace Road, Options: [(LOC), (ORG)]
    Answer: at Grace Road indicates this is a location or venue (LOC)    
    """

    dispute_exemplar_2 = """
    Their stay on top , though , may be short-lived as title rivals Essex , Derbyshire and Surrey all closed in on victory while Kent made up for lost time in their rain-affected match against Nottinghamshire .

    Entity Phrase: Essex, Options: [(LOC), (ORG)]
    Answer: As they are tital rivals, Essex is a sports team and not a location (ORG)

    Entity Phrase: Nottinghamshire, Options: [(LOC), (ORG)]
    Answer: As Nottinghamshire defeated Kent, this is a sports team not a location (ORG)
    """
    dispute_exemplars = [dispute_exemplar_1, dispute_exemplar_2]


In [36]:
politics_labels = ['O', 'B-country', 'B-politician', 'I-politician', 'B-election', 'I-election', 'B-person', 'I-person', 'B-organisation', 'I-organisation', 'B-location', 'B-misc', 'I-location', 'I-country', 'I-misc', 'B-politicalparty', 'I-politicalparty', 'B-event', 'I-event']
science_labels = ['O', 'B-scientist', 'I-scientist', 'B-person', 'I-person', 'B-university', 'I-university', 'B-organisation', 'I-organisation', 'B-country', 'I-country', 'B-location', 'I-location', 'B-discipline', 'I-discipline', 'B-enzyme', 'I-enzyme', 'B-protein', 'I-protein', 'B-chemicalelement', 'I-chemicalelement', 'B-chemicalcompound', 'I-chemicalcompound', 'B-astronomicalobject', 'I-astronomicalobject', 'B-academicjournal', 'I-academicjournal', 'B-event', 'I-event', 'B-theory', 'I-theory', 'B-award', 'I-award', 'B-misc', 'I-misc']
music_labels = ['O', 'B-musicgenre', 'I-musicgenre', 'B-song', 'I-song', 'B-band', 'I-band', 'B-album', 'I-album', 'B-musicalartist', 'I-musicalartist', 'B-musicalinstrument', 'I-musicalinstrument', 'B-award', 'I-award', 'B-event', 'I-event', 'B-country', 'I-country', 'B-location', 'I-location', 'B-organisation', 'I-organisation', 'B-person', 'I-person', 'B-misc', 'I-misc']
literature_labels = ["O", "B-book", "I-book", "B-writer", "I-writer", "B-award", "I-award", "B-poem", "I-poem", "B-event", "I-event", "B-magazine", "I-magazine", "B-literarygenre", "I-literarygenre", 'B-country', 'I-country', "B-person", "I-person", "B-location", "I-location", 'B-organisation', 'I-organisation', 'B-misc', 'I-misc']
ai_labels = ["O", "B-field", "I-field", "B-task", "I-task", "B-product", "I-product", "B-algorithm", "I-algorithm", "B-researcher", "I-researcher", "B-metrics", "I-metrics", "B-programlang", "I-programlang", "B-conference", "I-conference", "B-university", "I-university", "B-country", "I-country", "B-person", "I-person", "B-organisation", "I-organisation", "B-location", "I-location", "B-misc", "I-misc"]

domain2labels = {"politics": politics_labels, "science": science_labels, "music": music_labels, "literature": literature_labels, "ai": ai_labels}
def read_ner(datapath):
    inputs_text, labels = [], []
    with open(datapath, "r") as fr:
        token_list, label_list = [], []
        for i, line in enumerate(fr):
            #print(line)
            line = line.strip()
            if line == "":
                inputs_text.append(" ".join(token_list).replace(" .","."))
                labels.append(label_list)

                token_list, label_list = [], []
                continue
            
            splits = line.split("\t")
            #print(splits)
            token = splits[0]
            label = splits[1]
            #print()

            #subs_ = auto_tokenizer.tokenize(token)
            #if len(subs_) > 0:
            label_list.append(label)
            token_list.append(token)
            #else:
            #   print("length of subwords for %s is zero; its label is %s" % (token, label))

    return inputs_text, labels


In [37]:
inputs, labels=read_ner("data_promptner/data/CrossNER/ner_data/conll2003/dev.txt")

In [39]:
print(inputs[0], labels[0])

CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY. ['O', 'O', 'B-organisation', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [43]:
class BerocConfig:

    defn="""Vous êtes un système expert en Reconnaissance d'Entités Nommées (REN).
    Votre tâche consiste à accepter du Texte en entrée et à extraire les entités nommées exact (extrait de text)."""


    cot="""Les entités représentent des informations médicales spécifiques d'un dossier patient, incluant poids, âge, date de naissance, score PS, pathologie, localisation de la pathologie, latéralité, traitement, biomarqueur et résultats du biomarqueur.
    Ci-dessous se trouvent les définitions de chaque étiquette pour vous aider à comprendre quel type d'entités nommées extraire pour chaque étiquette.
    Supposez que ces définitions sont écrites par un expert et suivez-les attentivement.

    POIDS_PATIENT : Poids du patient, généralement en kilogrammes ou en livres.
    AGE_PATIENT : Âge du patient, habituellement en années.
    DATE_NAISSANCE_PATIENT : Date de naissance du patient, formatée en JJ\/MM\/AAAA ou un format similaire.
    SCORE_PS : Score de performance, indiquant le bien-être général et les capacités du patient.
    PATHOLOGIE : Type ou description de la pathologie diagnostiquée chez le patient.
    LOCALISATION_PATHOLOGIE : Localisation spécifique de la pathologie dans le corps du patient, par exemple, sein, poumon, os.
    LATERALITE : Indique le côté du corps affecté par la pathologie, par exemple, droite ou gauche.
    TRAITEMENT : Détails du traitement administré au patient, peut inclure le type, la durée et la fréquence.
    BIOMARQUEUR : Indicateur biologique mesurable et évaluable qui reflète l'état de la maladie, le pronostic ou la réponse au traitement.
    BIOMARQUEUR_RESULTS : Résultats ou valeurs quantitatives associées au biomarqueur, indiquant l'état ou la progression de la maladie.

    Si un segment n'est pas une entité, étiquetez-le : `==NONE==`."""

    examplair1="""Monsieur Dupont, né le 15\/04\/1962, pèse 78 kg et a été diagnostiqué avec un carcinome hépatocellulaire au lobe droit du foie.
    Réponse :
    1. 78 kg | True | POIDS_PATIENT | indique le poids du patient
    2. lobe droit du foie | True | LOCALISATION_PATHOLOGIE | précise la localisation de la pathologie"""

    examplair2= """La patiente, âgée de 55 ans, présente un PS de 2 et souffre d'un glioblastome au niveau de l'hémisphère cérébral gauche.
    Réponse :
    1. 55 ans | True | AGE_PATIENT | indique l'âge de la patiente
    2. PS de 2 | True | SCORE_PS | évalue l'état général de la patiente
    3. hémisphère cérébral gauche | True | LOCALISATION_PATHOLOGIE | précise la localisation de la pathologie dans le cerveau"""

    examplair3= """Né le 03\/08\/1970, le patient a un lymphome non hodgkinien situé dans les ganglions lymphatiques axillaires droits avec un score PS de 1.
    Réponse :
    1. ganglions lymphatiques axillaires droits | True | LOCALISATION_PATHOLOGIE | identifie la localisation spécifique du lymphome
    2. score PS de 1 | True | SCORE_PS | indique un bon état général du patient"""

    examplair4= """ Le dossier du patient, traité par le Dr. Leroy, indique un poids de 80 kg, né le 10\/02\/1965, souffrant d'un adénocarcinome au niveau du poumon gauche.
    Réponse :
    1. Dr. Leroy | False | ==NONE== | est le nom d'un médecin, pas une entité médicale pertinente
    2. 80 kg | True | POIDS_PATIENT | indique le poids du patient
    3. poumon gauche | True | LOCALISATION_PATHOLOGIE | précise la localisation de la pathologie"""

    examplair5= """" Madame Dupont, 72 ans, a consulté pour une douleur persistante au bras droit, avec un historique de lymphome etf PS de 1.
    Réponse :
    1. douleur persistante au bras droit | False | ==NONE== | est un symptôme, pas une entité médicale spécifique selon la configuration
    2. 72 ans | True | AGE_PATIENT | indique l'âge de la patiente
    3. PS de 1 | True | SCORE_PS | indique le score de performance de la patiente"""


    q= """Utilisez uniquement ce format de sortie dans l'exemple ci-dessus.
    Ne produisez rien d'autre que des entités dans ce format de sortie.
    Sortez les entités dans l'ordre où elles apparaissent dans le paragraphe d'entrée, indépendamment de l'étiquette."""

    paragraphe="""{paragraph}\nRéponse :"""

    exemplairs = [examplair4, examplair4, examplair4,examplair4,examplair5]

    def __init__(self, paragraphe=paragraphe,exemplairs=exemplairs,cot=cot,q=q,defn=defn):
        self.paragraph=paragraphe.fromat(paragraphe=paragraphe)
        self.exemplars=exemplairs
        self.cot=cot
        self.q=q
        self.defn=defn


In [ ]:
beroc_config = BerocConfig()
beroc_config.

In [ ]:
task = self.defn + "\n" + self.exemplar_task + f" '{self.para}' \nAnswer:"
output = self.model_fn(task)

In [41]:
p="""Médecin traitant : Dr RUISSEAU
    Lettre de liaison éditée et remise au patient le 21\/01\/2018

    Points clés du séjour
    Mme JAVERT Corine âgée de 57 ans, est hospitalisée dans le service d'endocrinologie pour la découverte d'un diabète insipide devant un syndrome polyuro-polydipsique (4 à 6 L\/ jour) et une natrémie maximale 149 mmol\/L.
    L'introduction d'un traitement par minirin 60ug matin et coucher permettant une franche diminution du syndrome polyuro polydipsique. Natrémie hebdomadaire pendant 1 mois pour adapter le traitement.

    Sortie : retour au domicile

    [Suivi prévu :
    Echographie thyroïdienne à prévoir par la patiente
    IRM hypophysaire 15\/01\/2018 à 7h30 à l'hopital Tadennec
    TDM TAP 15\/01\/2018 à 8h à l'hôpital Tadennec
    consultation endocrinologique avec un interne dans 1 mois, la patiente recevra une convocation
    consultation endocrinologique avec Dr CArole Mui le 15\/01\/2018 à 15h30

    Modifications thérapeutiques
    z Traitement à l'entrée :
    Traitement de sortie:
    5 - introduction minirin 60 Hg matin et à
    - SERETIDE 500 diskus, 2 coucher $ bouffées matin
    - SERETIDE 500 diskus, 2 $
    - TAMOXIFENE 20 mg, cpr voie bouffées matin £ orale
    - TAMOXIFENE 20 mg, cpr voie

    Nom : JAVERT CORINE NÉ(E)
    Page 2 de 5
    Né(e) le : TRINEAU 25\/11\/1961"""

In [46]:
paragraph="""{para}\nRéponse :"""

In [47]:
paragraph.format(para=p)

"Médecin traitant : Dr RUISSEAU\n    Lettre de liaison éditée et remise au patient le 21\\/01\\/2018\n\n    Points clés du séjour\n    Mme JAVERT Corine âgée de 57 ans, est hospitalisée dans le service d'endocrinologie pour la découverte d'un diabète insipide devant un syndrome polyuro-polydipsique (4 à 6 L\\/ jour) et une natrémie maximale 149 mmol\\/L.\n    L'introduction d'un traitement par minirin 60ug matin et coucher permettant une franche diminution du syndrome polyuro polydipsique. Natrémie hebdomadaire pendant 1 mois pour adapter le traitement.\n\n    Sortie : retour au domicile\n\n    [Suivi prévu :\n    Echographie thyroïdienne à prévoir par la patiente\n    IRM hypophysaire 15\\/01\\/2018 à 7h30 à l'hopital Tadennec\n    TDM TAP 15\\/01\\/2018 à 8h à l'hôpital Tadennec\n    consultation endocrinologique avec un interne dans 1 mois, la patiente recevra une convocation\n    consultation endocrinologique avec Dr CArole Mui le 15\\/01\\/2018 à 15h30\n\n    Modifications thérape

In [48]:
df=pd.read_excel("Exemples.xlsx")
df=df.drop(columns=["Unnamed: 7"])

FileNotFoundError: [Errno 2] No such file or directory: 'Exemples.xlsx'